In [1]:
import pandas as pd
import numpy as np

In [2]:
dt = pd.read_csv('tir.txt',header=None,
                 names=['x','y','v','can','angle'],na_values='None')

In [3]:
dtrg = dt[dt['angle'].notna()].drop(columns=['can'])
dtrg.shape

(119, 4)

In [4]:
dtrg.head()

,x,y,v,angle
3,17,1,16,24.261790
4,18,5,18,33.984204
5,20,-1,15,26.160026
8,14,0,18,12.540177
10,6,0,17,5.875763


In [5]:
ths = 90
avg = dtrg.iloc[:ths,:-1].mean()
std  = dtrg.iloc[:ths,:-1].std()
l = list(dtrg.columns.tolist())[:-1]
dtrg[l] = (dtrg.iloc[:,:-1] - avg) / std
dtrg.head()

,x,y,v,angle
3,0.009204,0.417326,-0.418675,24.261790
4,0.112747,1.609688,0.214615,33.984204
5,0.319834,-0.178854,-0.735320,26.160026
8,-0.301426,0.119236,0.214615,12.540177
10,-1.129773,0.119236,-0.102030,5.875763


## Regressió

In [6]:
from sklearn.metrics import r2_score
 
def regression(nom, rgs, ths):
    print(nom)
    rgs.fit(dtrg.iloc[:ths,:-1],dtrg.iloc[:ths,-1])

    preds = pd.Series(rgs.predict(dtrg.iloc[ths:,:-1]),name='preds')
    reals = pd.Series(dtrg.iloc[ths:,-1],name='reals')
    reals.index = range(dtrg.shape[0] - ths)
    
    print('r2:', round(r2_score(reals, preds), 3))

In [7]:
from sklearn.tree import DecisionTreeRegressor
regression('DTs:', DecisionTreeRegressor(), ths)

DTs:
r2: 0.927


In [8]:
from sklearn.neural_network import MLPRegressor
model = MLPRegressor(hidden_layer_sizes=(50,10,5),max_iter=1000)
regression('MLP:', model, ths)
model.n_iter_

MLP:
r2: 0.974


/usr/local/lib/python3.8/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:582: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(


1000

## Keras

In [9]:
import warnings  
with warnings.catch_warnings():  
    warnings.filterwarnings("ignore",category=FutureWarning)
    from tensorflow import keras
    from tensorflow.keras.datasets import mnist
    from tensorflow.keras.models import Sequential
    from tensorflow.keras.layers import Dense, Dropout
    from tensorflow.keras.optimizers import Adam

In [10]:
model = Sequential()
model.add(Dense(50, activation='relu', input_shape=(3,)))
model.add(Dropout(0.2))
model.add(Dense(20, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(5, activation='relu'))
model.add(Dense(1, activation='linear'))

In [11]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 50)                200       
_________________________________________________________________
dropout (Dropout)            (None, 50)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 20)                1020      
_________________________________________________________________
dropout_1 (Dropout)          (None, 20)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 5)                 105       
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 6         
Total params: 1,331
Trainable params: 1,331
Non-trainable params: 0
______________________________________________________

In [12]:
model.compile(loss='mse',
              optimizer=Adam(),
              metrics=['mean_squared_error'])

In [13]:
model.fit(dtrg.iloc[:ths,:-1], dtrg.iloc[:ths,-1], batch_size=32, epochs=1000)

Epoch 1/1000
3/3 [==============================] - 0s 2ms/step - loss: 663.3080 - mean_squared_error: 663.3080
Epoch 2/1000
3/3 [==============================] - 0s 1ms/step - loss: 659.9340 - mean_squared_error: 659.9340
Epoch 3/1000
3/3 [==============================] - 0s 1ms/step - loss: 658.5391 - mean_squared_error: 658.5391
Epoch 4/1000
3/3 [==============================] - 0s 1ms/step - loss: 657.1054 - mean_squared_error: 657.1054
Epoch 5/1000
3/3 [==============================] - 0s 1ms/step - loss: 649.3446 - mean_squared_error: 649.3446
Epoch 6/1000
3/3 [==============================] - 0s 1ms/step - loss: 646.6278 - mean_squared_error: 646.6278
Epoch 7/1000
3/3 [==============================] - 0s 1ms/step - loss: 644.5555 - mean_squared_error: 644.5555
Epoch 8/1000
3/3 [==============================] - 0s 1ms/step - loss: 641.6506 - mean_squared_error: 641.6506
Epoch 9/1000
3/3 [==============================] - 0s 1ms/step - loss: 634.4711 - mean_squared_error: 6

3/3 [==============================] - 0s 1ms/step - loss: 111.4609 - mean_squared_error: 111.4609
Epoch 74/1000
3/3 [==============================] - 0s 1ms/step - loss: 118.0736 - mean_squared_error: 118.0736
Epoch 75/1000
3/3 [==============================] - 0s 1ms/step - loss: 135.0513 - mean_squared_error: 135.0513
Epoch 76/1000
3/3 [==============================] - ETA: 0s - loss: 163.9380 - mean_squared_error: 163.93 - 0s 1ms/step - loss: 118.5137 - mean_squared_error: 118.5137
Epoch 77/1000
3/3 [==============================] - 0s 1ms/step - loss: 119.2819 - mean_squared_error: 119.2819
Epoch 78/1000
3/3 [==============================] - 0s 1ms/step - loss: 112.1179 - mean_squared_error: 112.1179
Epoch 79/1000
3/3 [==============================] - 0s 1ms/step - loss: 113.1056 - mean_squared_error: 113.1056
Epoch 80/1000
3/3 [==============================] - 0s 1ms/step - loss: 115.4679 - mean_squared_error: 115.4679
Epoch 81/1000
3/3 [==============================] - 0

3/3 [==============================] - 0s 1ms/step - loss: 111.4499 - mean_squared_error: 111.4499
Epoch 145/1000
3/3 [==============================] - 0s 913us/step - loss: 108.7058 - mean_squared_error: 108.7058
Epoch 146/1000
3/3 [==============================] - 0s 1ms/step - loss: 107.3391 - mean_squared_error: 107.3391
Epoch 147/1000
3/3 [==============================] - 0s 1ms/step - loss: 119.1516 - mean_squared_error: 119.1516
Epoch 148/1000
3/3 [==============================] - 0s 1ms/step - loss: 108.6121 - mean_squared_error: 108.6121
Epoch 149/1000
3/3 [==============================] - 0s 1ms/step - loss: 98.1778 - mean_squared_error: 98.1778
Epoch 150/1000
3/3 [==============================] - 0s 1ms/step - loss: 98.7342 - mean_squared_error: 98.7342
Epoch 151/1000
3/3 [==============================] - 0s 1ms/step - loss: 107.1812 - mean_squared_error: 107.1812
Epoch 152/1000
3/3 [==============================] - 0s 1ms/step - loss: 104.3465 - mean_squared_error: 

3/3 [==============================] - 0s 1ms/step - loss: 117.1095 - mean_squared_error: 117.1095
Epoch 216/1000
3/3 [==============================] - 0s 1ms/step - loss: 102.8146 - mean_squared_error: 102.8146
Epoch 217/1000
3/3 [==============================] - 0s 1ms/step - loss: 98.4771 - mean_squared_error: 98.4771
Epoch 218/1000
3/3 [==============================] - 0s 1ms/step - loss: 104.6217 - mean_squared_error: 104.6217
Epoch 219/1000
3/3 [==============================] - 0s 1ms/step - loss: 101.5362 - mean_squared_error: 101.5362
Epoch 220/1000
3/3 [==============================] - 0s 1ms/step - loss: 108.6861 - mean_squared_error: 108.6861
Epoch 221/1000
3/3 [==============================] - 0s 1ms/step - loss: 104.5223 - mean_squared_error: 104.5223
Epoch 222/1000
3/3 [==============================] - 0s 985us/step - loss: 108.1859 - mean_squared_error: 108.1859
Epoch 223/1000
3/3 [==============================] - 0s 946us/step - loss: 105.3795 - mean_squared_err

3/3 [==============================] - 0s 1ms/step - loss: 100.5479 - mean_squared_error: 100.5479
Epoch 287/1000
3/3 [==============================] - 0s 1ms/step - loss: 86.6070 - mean_squared_error: 86.6070
Epoch 288/1000
3/3 [==============================] - 0s 1ms/step - loss: 80.8733 - mean_squared_error: 80.8733
Epoch 289/1000
3/3 [==============================] - 0s 1ms/step - loss: 89.9375 - mean_squared_error: 89.9375
Epoch 290/1000
3/3 [==============================] - 0s 935us/step - loss: 76.1534 - mean_squared_error: 76.1534
Epoch 291/1000
3/3 [==============================] - 0s 1ms/step - loss: 80.4548 - mean_squared_error: 80.4548
Epoch 292/1000
3/3 [==============================] - 0s 939us/step - loss: 76.2806 - mean_squared_error: 76.2806
Epoch 293/1000
3/3 [==============================] - 0s 1ms/step - loss: 89.1404 - mean_squared_error: 89.1404
Epoch 294/1000
3/3 [==============================] - 0s 1ms/step - loss: 79.0183 - mean_squared_error: 79.0183
E

...

3/3 [==============================] - 0s 1ms/step - loss: 27.7976 - mean_squared_error: 27.7976
Epoch 650/1000
3/3 [==============================] - 0s 1ms/step - loss: 20.3551 - mean_squared_error: 20.3551
Epoch 651/1000
3/3 [==============================] - 0s 1ms/step - loss: 25.0515 - mean_squared_error: 25.0515
Epoch 652/1000
3/3 [==============================] - 0s 955us/step - loss: 37.6448 - mean_squared_error: 37.6448
Epoch 653/1000
3/3 [==============================] - 0s 1ms/step - loss: 23.7707 - mean_squared_error: 23.7707
Epoch 654/1000
3/3 [==============================] - 0s 949us/step - loss: 31.3061 - mean_squared_error: 31.3061
Epoch 655/1000
3/3 [==============================] - 0s 1ms/step - loss: 33.6785 - mean_squared_error: 33.6785
Epoch 656/1000
3/3 [==============================] - 0s 1ms/step - loss: 34.7096 - mean_squared_error: 34.7096
Epoch 657/1000
3/3 [==============================] - 0s 937us/step - loss: 28.0951 - mean_squared_error: 28.0951
E

3/3 [==============================] - 0s 1ms/step - loss: 23.5470 - mean_squared_error: 23.5470
Epoch 722/1000
3/3 [==============================] - 0s 1ms/step - loss: 25.7312 - mean_squared_error: 25.7312
Epoch 723/1000
3/3 [==============================] - 0s 1ms/step - loss: 28.0285 - mean_squared_error: 28.0285
Epoch 724/1000
3/3 [==============================] - 0s 962us/step - loss: 22.6094 - mean_squared_error: 22.6094
Epoch 725/1000
3/3 [==============================] - 0s 1ms/step - loss: 36.0352 - mean_squared_error: 36.0352
Epoch 726/1000
3/3 [==============================] - 0s 1ms/step - loss: 30.5026 - mean_squared_error: 30.5026
Epoch 727/1000
3/3 [==============================] - 0s 1ms/step - loss: 21.7453 - mean_squared_error: 21.7453
Epoch 728/1000
3/3 [==============================] - 0s 1ms/step - loss: 31.4355 - mean_squared_error: 31.4355
Epoch 729/1000
3/3 [==============================] - 0s 1ms/step - loss: 37.3563 - mean_squared_error: 37.3563
Epoch

3/3 [==============================] - 0s 1ms/step - loss: 19.6723 - mean_squared_error: 19.6723
Epoch 794/1000
3/3 [==============================] - 0s 960us/step - loss: 26.7324 - mean_squared_error: 26.7324
Epoch 795/1000
3/3 [==============================] - 0s 1ms/step - loss: 36.2477 - mean_squared_error: 36.2477
Epoch 796/1000
3/3 [==============================] - 0s 1ms/step - loss: 25.2606 - mean_squared_error: 25.2606
Epoch 797/1000
3/3 [==============================] - 0s 1ms/step - loss: 27.2188 - mean_squared_error: 27.2188
Epoch 798/1000
3/3 [==============================] - 0s 1ms/step - loss: 43.5239 - mean_squared_error: 43.5239
Epoch 799/1000
3/3 [==============================] - 0s 960us/step - loss: 26.9695 - mean_squared_error: 26.9695
Epoch 800/1000
3/3 [==============================] - 0s 1ms/step - loss: 31.5296 - mean_squared_error: 31.5296
Epoch 801/1000
3/3 [==============================] - 0s 991us/step - loss: 31.1098 - mean_squared_error: 31.1098
E

3/3 [==============================] - 0s 1ms/step - loss: 27.1218 - mean_squared_error: 27.1218
Epoch 866/1000
3/3 [==============================] - 0s 1ms/step - loss: 14.0397 - mean_squared_error: 14.0397
Epoch 867/1000
3/3 [==============================] - 0s 988us/step - loss: 24.6693 - mean_squared_error: 24.6693
Epoch 868/1000
3/3 [==============================] - 0s 1ms/step - loss: 21.3605 - mean_squared_error: 21.3605
Epoch 869/1000
3/3 [==============================] - 0s 1ms/step - loss: 25.5973 - mean_squared_error: 25.5973
Epoch 870/1000
3/3 [==============================] - 0s 977us/step - loss: 29.4447 - mean_squared_error: 29.4447
Epoch 871/1000
3/3 [==============================] - 0s 1ms/step - loss: 22.2298 - mean_squared_error: 22.2298
Epoch 872/1000
3/3 [==============================] - 0s 1ms/step - loss: 18.8006 - mean_squared_error: 18.8006
Epoch 873/1000
3/3 [==============================] - 0s 920us/step - loss: 18.0218 - mean_squared_error: 18.0218
E

3/3 [==============================] - 0s 1ms/step - loss: 24.4402 - mean_squared_error: 24.4402
Epoch 939/1000
3/3 [==============================] - 0s 1ms/step - loss: 28.8663 - mean_squared_error: 28.8663
Epoch 940/1000
3/3 [==============================] - 0s 1ms/step - loss: 33.9359 - mean_squared_error: 33.9359
Epoch 941/1000
3/3 [==============================] - 0s 1ms/step - loss: 25.1508 - mean_squared_error: 25.1508
Epoch 942/1000
3/3 [==============================] - 0s 1ms/step - loss: 26.4108 - mean_squared_error: 26.4108
Epoch 943/1000
3/3 [==============================] - 0s 941us/step - loss: 23.3429 - mean_squared_error: 23.3429
Epoch 944/1000
3/3 [==============================] - 0s 1ms/step - loss: 25.4636 - mean_squared_error: 25.4636
Epoch 945/1000
3/3 [==============================] - 0s 1ms/step - loss: 26.8346 - mean_squared_error: 26.8346
Epoch 946/1000
3/3 [==============================] - 0s 1ms/step - loss: 23.3948 - mean_squared_error: 23.3948
Epoch

In [14]:
model.evaluate(dtrg.iloc[ths:,:-1], dtrg.iloc[ths:,-1], verbose=0)

[4.871474266052246, 4.871474266052246]

In [15]:
preds = pd.Series(model.predict(dtrg.iloc[ths:,:-1]).T[0],name='preds')
reals = pd.Series(dtrg.iloc[ths:,-1],name='reals')
reals.index = range(dtrg.shape[0] - ths)
print('r2:', round(r2_score(reals, preds), 3))

r2: 0.986


In [16]:
# https://docs.unity3d.com/Packages/com.unity.barracuda@1.0/manual/Exporting.html
# convert model to ONNX
import keras2onnx
onnx_model = keras2onnx.convert_keras(model,       # keras model
                         name="tir",               # the converted ONNX model internal name                     
                         target_opset=9,           # the ONNX version to export the model to
                         channel_first_inputs=None # which inputs to transpose from NHWC to NCHW
                         )

keras2onnx.save_model(onnx_model, "tir.onnx")

tf executing eager_mode: True
tf.keras model eager_mode: False
The ONNX operator number change on the optimization: 18 -> 11
The maximum opset needed by this model is only 7.


## Example Rest

In [17]:
#test
tst = dtrg.iloc[-2:]
tst

,x,y,v,angle
196,-1.026230,-1.371215,-0.418675,-28.409901
197,-0.301426,0.715417,0.214615,21.129271


In [18]:
pd.Series(model.predict(dtrg.iloc[-2:,:-1]).T[0],name='preds',index=[196,197])

196   -24.272972
197    19.843548
Name: preds, dtype: float32

In [19]:
s = pd.Series(model.predict(dtrg.iloc[-2:,:-1]).T[0],name='preds',index=[196,197])
s

196   -24.272972
197    19.843548
Name: preds, dtype: float32

In [20]:
pd.concat([tst,s],axis=1)

,x,y,v,angle,preds
196,-1.026230,-1.371215,-0.418675,-28.409901,-24.272972
197,-0.301426,0.715417,0.214615,21.129271,19.843548
